In [2]:
import warnings

# 경고 무시
warnings.filterwarnings('ignore')

In [3]:
import os

# ./cache/ 경로 설정
os.environ["HF_HOME"] = "./cache/"

# Sample Data

In [4]:
from langchain_core.documents import Document

texts = [
    "안녕, 만나서 반가워.",
    "LangChain simplifies the process of building applications with large language models",
    "랭체인 한국어 튜토리얼은 LangChain의 공식 문서, cookbook 및 다양한 실용 예제를 바탕으로 하여 사용자가 LangChain을 더 쉽고 효과적으로 활용할 수 있도록 구성되어 있습니다. ",
    "LangChain은 초거대 언어모델로 애플리케이션을 구축하는 과정을 단순화합니다.",
    "Retrieval-Augmented Generation (RAG) is an effective technique for improving AI responses.",
]

In [5]:
!pip install -qU langchain_huggingface

In [7]:
from langchain_huggingface.embeddings import HuggingFaceEndpointEmbeddings

model_name = "intfloat/multilingual-e5-large-instruct"

hf_embeddings = HuggingFaceEndpointEmbeddings(
    model = model_name,
    task = "feature-extraction",
    huggingfacehub_api_token = "huggingface_api_key",
)

In [8]:
%%time

# Document Embedding 수행
embedded_documents = hf_embeddings.embed_documents(texts)

CPU times: user 27 ms, sys: 1.22 ms, total: 28.2 ms
Wall time: 239 ms


In [9]:
print("[HuggingFace Endpoint Embedding]")
print(f"Model: \t\t{model_name}")
print(f"Dimension: \t{len(embedded_documents[0])}")

[HuggingFace Endpoint Embedding]
Model: 		intfloat/multilingual-e5-large-instruct
Dimension: 	1024


In [10]:
# Document Embedding 수행
embedded_query = hf_embeddings.embed_query("LangChain 에 대해서 알려주세요.")

In [11]:
len(embedded_query)

1024

# 유사도 계산
## 벡터 내적을 통한 유사도 계산
벡터 내적(dot product)을 사용하여 유사도를 계산합니다.

유사도 계산 공식:

$$ \text{similarities} = \mathbf{query} \cdot \mathbf{documents}^T $$

## 벡터 내적의 수학적 의미

벡터 내적 정의

벡터 $\mathbf{a}$와 $\mathbf{b}$의 내적은 다음과 같이 정의됩니다:
$$ \mathbf{a} \cdot \mathbf{b} = \sum_{i=1}^{n} a_i b_i $$

## 코사인 유사도와의 관계

벡터 내적은 다음과 같은 성질을 가집니다.
$$ \mathbf{a} \cdot \mathbf{b} = |\mathbf{a}| |\mathbf{b}| \cos \theta $$

여기서,
$|\mathbf{a}|$와 $|\mathbf{b}|$는 각각 벡터 $\mathbf{a}$와 $\mathbf{b}$의 크기(노름, Euclidean norm)입니다.
$\theta$는 두 벡터 사이의 각도입니다.
$\cos \theta$는 두 벡터 사이의 코사인 유사도입니다.

## 벡터 내적의 유사도 해석
내적 값이 클수록 (양의 큰 값일수록),
두 벡터의 크기($|\mathbf{a}|$와 $|\mathbf{b}|$)가 크고,
두 벡터 사이의 각도($\theta$)가 작으며 ($\cos \theta$가 1에 가까움),

이는 두 벡터가 유사한 방향을 가리키고, 크기가 클수록 더 유사하다는 것을 의미합니다.

## 벡터의 크기(노름) 계산

Euclidean norm 정의
벡터 $\mathbf{a} = [a_1, a_2, \ldots, a_n]$에 대해, Euclidean norm $|\mathbf{a}|$는 다음과 같이 정의됩니다:
$$ |\mathbf{a}| = \sqrt{a_1^2 + a_2^2 + \cdots + a_n^2} $$

In [13]:
import numpy as np

np.array(embedded_query) @ np.array(embedded_documents).T

array([0.84186319, 0.86502318, 0.86470304, 0.89564882, 0.76847344])

In [14]:
sorted_idx = (np.array(embedded_query) @ np.array(embedded_documents).T).argsort()[::-1]
sorted_idx

array([3, 1, 2, 0, 4])

In [15]:
for i, idx in enumerate(sorted_idx):
    print(f"[{i}] {texts[idx]}")
    print()

[0] LangChain은 초거대 언어모델로 애플리케이션을 구축하는 과정을 단순화합니다.

[1] LangChain simplifies the process of building applications with large language models

[2] 랭체인 한국어 튜토리얼은 LangChain의 공식 문서, cookbook 및 다양한 실용 예제를 바탕으로 하여 사용자가 LangChain을 더 쉽고 효과적으로 활용할 수 있도록 구성되어 있습니다. 

[3] 안녕, 만나서 반가워.

[4] Retrieval-Augmented Generation (RAG) is an effective technique for improving AI responses.



# Localization

In [17]:
from langchain_huggingface.embeddings import HuggingFaceEmbeddings

model_name = "intfloat/multilingual-e5-large-instruct"

hf_embeddings = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs={"device": "cpu"},  # cuda, cpu
    encode_kwargs={"normalize_embeddings": True},
)

In [18]:
%time

embedded_documents1 = hf_embeddings.embed_documents(texts)

CPU times: user 11 µs, sys: 4 µs, total: 15 µs
Wall time: 19.6 µs


In [19]:
print(f"Model: \t\t{model_name}")
print(f"Dimension: \t{len(embedded_documents[0])}")

Model: 		intfloat/multilingual-e5-large-instruct
Dimension: 	1024


In [20]:
from langchain_huggingface.embeddings import HuggingFaceEmbeddings

model_name = "BAAI/bge-m3"

model_kwarge = {"device": "cpu"}
encode_kwargs = {"normalize_embeddings": True}

hf_embeddings = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwarge,
    encode_kwargs=encode_kwargs,
)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/123 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/15.8k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/54.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/687 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/444 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/964 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/191 [00:00<?, ?B/s]

In [21]:
%time
# Document
embedded_documents = hf_embeddings.embed_documents(texts)

CPU times: user 3 µs, sys: 2 µs, total: 5 µs
Wall time: 14.8 µs


In [22]:
print(f"Model: \t\t{model_name}")
print(f"Dimension: \t{len(embedded_documents[0])}")

Model: 		BAAI/bge-m3
Dimension: 	1024


In [23]:
import numpy as np

embedded_query = hf_embeddings.embed_query("LangChain 에 대해서 알려주세요.")
embedded_documents = hf_embeddings.embed_documents(texts)

# 질문(embedded_query): LangChain 에 대해서 알려주세요.
np.array(embedded_query) @ np.array(embedded_documents).T

sorted_idx = (np.array(embedded_query) @ np.array(embedded_documents).T).argsort()[::-1]

print("[Query] LangChain 에 대해서 알려주세요.\n====================================")
print()
for i, idx in enumerate(sorted_idx):
    print(f"[{i}] {texts[idx]}")
    print()

[Query] LangChain 에 대해서 알려주세요.

[0] LangChain simplifies the process of building applications with large language models

[1] LangChain은 초거대 언어모델로 애플리케이션을 구축하는 과정을 단순화합니다.

[2] 랭체인 한국어 튜토리얼은 LangChain의 공식 문서, cookbook 및 다양한 실용 예제를 바탕으로 하여 사용자가 LangChain을 더 쉽고 효과적으로 활용할 수 있도록 구성되어 있습니다. 

[3] 안녕, 만나서 반가워.

[4] Retrieval-Augmented Generation (RAG) is an effective technique for improving AI responses.



# FlagEmbedding

- Dense Vector: BGE-M3의 다국어, 다중 작업 능력을 기반으로 함
- Lexical weight를 활용한 sparse embedding으로 정확한 단어 매칭을 수행
- ColBERT의 multi-vector 접근법으로 문맥을 고려한 세밀한 매칭 수행

In [24]:
# FlagEmbedding 설치
!pip install -qU FlagEmbedding

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 161.8/161.8 kB 3.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 2.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 16.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 29.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 347.9/347.9 kB 18.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 172.0/172.0 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.2/119.2 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.4/45.4 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 22.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 31.3 MB/s eta 0:00:00
   

In [25]:
from FlagEmbedding import BGEM3FlagModel

model_name = "BAAI/bge-m3"

bge_embeddings = BGEM3FlagModel(
    model_name, use_fp16=True
)  # use_fp16을 True로 설정하면 약간의 성능 저하와 함께 계산 속도가 빨라짐

bge_embedded = bge_embeddings.encode(
    texts,
    batch_size = 12,
    max_length = 8192,  # 이렇게 긴 길이가 필요하지 않은 경우 더 작은 값을 설정하여 인코딩 프로세스의 속도를 높일 수 있음
)["dense_vecs"]

Fetching 30 files:   0%|          | 0/30 [00:00<?, ?it/s]

imgs/.DS_Store:   0%|          | 0.00/6.15k [00:00<?, ?B/s]

.gitattributes:   0%|          | 0.00/1.63k [00:00<?, ?B/s]

imgs/mkqa.jpg:   0%|          | 0.00/608k [00:00<?, ?B/s]

imgs/miracl.jpg:   0%|          | 0.00/576k [00:00<?, ?B/s]

...

model.onnx:   0%|          | 0.00/725k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

sparse_linear.pt:   0%|          | 0.00/3.52k [00:00<?, ?B/s]

onnx/tokenizer_config.json:   0%|          | 0.00/1.17k [00:00<?, ?B/s]

You're using a XLMRobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


In [26]:
bge_embedded.shape

(5, 1024)

In [27]:
print(f"Model: \t\t{model_name}")
print(f"Dimension: \t{len(embedded_documents[0])}")

Model: 		BAAI/bge-m3
Dimension: 	1024


# Lexical

In [28]:
from FlagEmbedding import BGEM3FlagModel

bge_flagmodel = BGEM3FlagModel(
    "BAAI/bge-m3", use_fp16=True
)  # use_fp16을 True로 설정하면 약간의 성능 저하와 함께 계산 속도가 빨라짐

bge_encoded = bge_flagmodel.encode(texts, return_dense = True)

Fetching 30 files:   0%|          | 0/30 [00:00<?, ?it/s]

You're using a XLMRobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


In [31]:
# 결과 출력(행, 열)
bge_encoded["dense_vecs"].shape

(5, 1024)

In [32]:
bge_flagmodel = BGEM3FlagModel(
    "BAAI/bge-m3", use_fp16=True
)  # use_fp16을 True로 설정하면 약간의 성능 저하와 함께 계산 속도가 빨라짐

bge_encoded = bge_flagmodel.encode(texts, return_sparse=True)

Fetching 30 files:   0%|          | 0/30 [00:00<?, ?it/s]

You're using a XLMRobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


In [33]:
lexical_scores1 = bge_flagmodel.compute_lexical_matching_score(
    bge_encoded["lexical_weights"][0], bge_encoded["lexical_weights"][0]
)
lexical_scores2 = bge_flagmodel.compute_lexical_matching_score(
    bge_encoded["lexical_weights"][0], bge_encoded["lexical_weights"][2]
)
# 0 <-> 0
print(lexical_scores1)
# 0 <-> 1
print(lexical_scores2)

0.3015604605898261
0


In [34]:
num_sentences = 5

# 유사도 계산 및 출력
for i in range(num_sentences):
  for j in range(num_sentences):
    lexical_score = bge_flagmodel.compute_lexical_matching_score(
        bge_encoded["lexical_weights"][i], bge_encoded["lexical_weights"][j]
    )
    print(f"{i} <==> {j}: {lexical_score:.4f}")

0 <==> 0: 0.3016
0 <==> 1: 0.0000
0 <==> 2: 0.0002
0 <==> 3: 0.0006
0 <==> 4: 0.0040
1 <==> 0: 0.0000
1 <==> 1: 0.5145
1 <==> 2: 0.1478
1 <==> 3: 0.2017
1 <==> 4: 0.0000
2 <==> 0: 0.0002
2 <==> 1: 0.1478
2 <==> 2: 0.5018
2 <==> 3: 0.1452
2 <==> 4: 0.0000
3 <==> 0: 0.0006
3 <==> 1: 0.2017
3 <==> 2: 0.1452
3 <==> 3: 0.5084
3 <==> 4: 0.0002
4 <==> 0: 0.0040
4 <==> 1: 0.0000
4 <==> 2: 0.0000
4 <==> 3: 0.0002
4 <==> 4: 0.8304


# ColBERT

#### 장점

- 토큰 수준의 세밀한 매칭이 가능
- 문맥을 고려한 임베딩을 생성
- 긴 문서에 대해서도 효과적으로 작동

In [35]:
bge_flagmodel = BGEM3FlagModel(
    "BAAI/bge-m3", use_fp16=True
)  # use_fp16을 True로 설정하면 약간의 성능 저하와 함께 계산 속도가 빨라집니다.

bge_encoded = bge_flagmodel.encode(texts, return_colbert_vecs=True)

Fetching 30 files:   0%|          | 0/30 [00:00<?, ?it/s]

You're using a XLMRobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


In [36]:
colbert_scores1 = bge_flagmodel.colbert_score(
    bge_encoded["colbert_vecs"][0], bge_encoded["colbert_vecs"][0]
)
colbert_scores2 = bge_flagmodel.colbert_score(
    bge_encoded["colbert_vecs"][0], bge_encoded["colbert_vecs"][1]
)
# 0 <-> 0
print(colbert_scores1)
# 0 <-> 1
print(colbert_scores2)

tensor(1.)
tensor(0.3748)


In [40]:
num_sentences = 5

# 유사도 계산 및 출력
for i in range(num_sentences):
    for j in range(num_sentences):
        colbert_score = bge_flagmodel.colbert_score(
            bge_encoded["colbert_vecs"][i], bge_encoded["colbert_vecs"][j]
        )
        print(f"{i} <==> {j} (ColBERT): {colbert_score:.4f}")

0 <==> 0 (ColBERT): 1.0000
0 <==> 1 (ColBERT): 0.3748
0 <==> 2 (ColBERT): 0.2793
0 <==> 3 (ColBERT): 0.3498
0 <==> 4 (ColBERT): 0.3649
1 <==> 0 (ColBERT): 0.3487
1 <==> 1 (ColBERT): 1.0000
1 <==> 2 (ColBERT): 0.5894
1 <==> 3 (ColBERT): 0.8961
1 <==> 4 (ColBERT): 0.4096
2 <==> 0 (ColBERT): 0.2190
2 <==> 1 (ColBERT): 0.5334
2 <==> 2 (ColBERT): 1.0000
2 <==> 3 (ColBERT): 0.5514
2 <==> 4 (ColBERT): 0.3236
3 <==> 0 (ColBERT): 0.2688
3 <==> 1 (ColBERT): 0.8746
3 <==> 2 (ColBERT): 0.5846
3 <==> 3 (ColBERT): 1.0000
3 <==> 4 (ColBERT): 0.3862
4 <==> 0 (ColBERT): 0.3413
4 <==> 1 (ColBERT): 0.4110
4 <==> 2 (ColBERT): 0.3499
4 <==> 3 (ColBERT): 0.3970
4 <==> 4 (ColBERT): 1.0000
